# Data Science Jobs Analyze


Group:
* Anchal
* Angela
* Larissa
* Fabio
* Felipe

This program extracts data from Indeed.ca website to analyze main caracteristics 
of Data Science Job opportunities in Toronto

In [1]:
#################### IMPORT LIBRARIES ####################

In [2]:
#import bs4
import numpy as np
import pandas as pd
import re 
import requests
import datetime
from bs4 import BeautifulSoup

In [3]:
#################### SEARCH ARGUMENTS ####################

In [4]:
input_job = "Data Scientist"
input_quote = False # add quotation marks("") to your input_job
input_city = "" # leave empty if input_city is not specified
input_state = "Canada"
sign = "+"
BASE_URL_indeed =  'https://ca.indeed.com/' #'http://www.indeed.com'


In [5]:
##### Function for Transform searching keywords #####
def transform(input,sign, quote = False):
    syntax = input.replace(" ", sign)
    if quote == True:
        syntax = ''.join(['"', syntax, '"'])
    return(syntax)

In [6]:
################# GENERATING BASE INDEED URL #################

In [7]:

if not input_city: # if (input_city is "")
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', input_state]
    url_indeed = ''.join(url_indeed_list)
else: # input_city is not ""
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', transform(input_city, sign), '%2C+', input_state]
    url_indeed = ''.join(url_indeed_list)

print(url_indeed)


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada


In [8]:
# get the HTML code from the URL
rawcode_indeed = requests.get(url_indeed)

# Choose "lxml" as parser
soup_indeed = BeautifulSoup(rawcode_indeed.text, "lxml")


In [9]:
# total number of results
num_total_indeed = soup_indeed.find(
                        id = 'searchCount').contents[0].split()[-2]

print("total results:")
print(soup_indeed.find(id = 'searchCount').contents[0])
print(soup_indeed.find(id = 'searchCount').contents[0].split()[-2])
num_total_indeed = re.sub("[^0-9]","", num_total_indeed) # remove non-numeric characters in the string
num_total_indeed = int(num_total_indeed)
#print(num_total_indeed)

# total number of pages
num_pages_indeed = int(np.ceil(num_total_indeed/11.0))
#print(num_pages_indeed)

# create an empty dataframe
df_base = pd.DataFrame()
# the date for today
now = datetime.datetime.now()
now_str = now.strftime("%m/%d/%Y")
now_str_name=now.strftime('%m%d%Y')

total results:

        Page 1 of 1,058 jobs
1,058


In [10]:
############ function to clean text before scraping ########### 

In [11]:
def text_cleaner(dirty_text):

    text = dirty_text

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception
   
    #print(text)
    text = text.decode('utf-8')
    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    text = text.lower().split()  # Go to lower case and split them apart
    
    
   # stop_words = set(stopwords.words("english")) # Filter out any stop words
   # text = [w for w in text if not w in stop_words]
    
    text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

######### CREATING DATA FRAME WITH ALL JOB OPPORTUNITIES URLS #####

In [12]:
######## functions ###########

def Salary(soup):
    try:
        Source = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'}).text
        # print(Source)
        Souce_first_letter = Source[0:1]
        # print (Souce_first_letter)
        if Souce_first_letter == '$':
            Salary = Source
            return Salary
        else:
            Salary = 'Not Informed'
            return Salary
    except:
        Salary = 'Not Informed'
        return Salary

def Job_Type(soup):
    try:
        Source = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'}).text
        # print(Source)
        Souce_first_letter = Source[0:1]
        # print (Souce_first_letter)
        if Souce_first_letter != '$':
            Job_Type = Source
            return Job_Type
        else:
            Job_Type = 'Not Informed'
            return Job_Type
    except:
        Job_Type = 'Not Informed'
        return Job_Type

    
def City(soup):
    try:
        Position = soup.find( attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
#         print (Position.text)
        Title_count = len(Position.text)
        Location_2 = soup.find('title')
#         print(Location_2.text)
        Final = Location_2.text.find('Indeed')
#         print (Final)
        City = Location_2.text[Title_count+3: Final-3].split(',')[0]
        return City
    except:
        City = 'Not Informed'
        return City

    
def Province(soup):
    try:
        Position = soup.find( attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
#         print (Position.text)
        Title_count = len(Position.text)
        Location_2 = soup.find('title')
#         print(Location_2.text)
        Final = Location_2.text.find('Indeed')
#         print (Final)
        Province = Location_2.text[Title_count+3: Final-3].split(', ')[1]
        return Province
    except:
        Province = 'Not Informed'
        return Province
    
def Position(soup):
    try:
        position_1 = soup_html.find(attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
        Position = position_1.text
        return Position
    except:
        Position = 'Not Informed'
        return Position
        
def find_skill(skill, jobReq ):
    if skill in jobReq:
        return True
    else:
        return False
    
def Job_requirements (soup):
    try:
        General_1 = soup.find( attrs = {'class' : 'jobsearch-JobComponent-description icl-u-xs-mt--md'})
        General = General_1.text
        return General
    except:
        General = 'Not Informed' 
        return General
    
def Company_Name(soup):
    try:
        Company_Name_1 = soup.find( attrs = {'class' : 'icl-u-lg-mr--sm icl-u-xs-mr--xs'})
        Company_name = Company_Name_1.text
        return Company_name
    except:
        Company_name = 'Not Informed'
        return Company_name       
        

In [13]:
########################################
##### Loop for all the total pages #####
########################################

# num_pages_indeed = 5
df_base = pd.DataFrame()

for i in range(1, num_pages_indeed+1):
    # generate the URL
    url = ''.join([url_indeed, '&start=', str(i*10)])
    #print(url)

    # get  LXML
    rawcode = requests.get(url)
    soup_lxml = BeautifulSoup(rawcode.text, "lxml")
    
    
    #Extracting the Salary


    # pick out all the "div" with "class="job-row"
    divs = soup_lxml.findAll("div")
    job_divs = [jp for jp in divs if not jp.get('class') is None
                    and 'row' in jp.get('class')]

    
    for job in job_divs:
        try:
            
            ix = job.index
            # job id
            id = job.get('data-jk', None)
            #print(id)
            
            # job link related to job id
            link = BASE_URL_indeed + '/viewjob'+ '?jk=' + id
            #link = 'https://ca.indeed.com/Data-Scientist-jobs-in-Canada?vjk=' + id
            #print(link)
            
            # job title
#             title = job.find('a', attrs={'data-tn-element': 'jobTitle'}).attrs['title']
            #print(title)
            
            #sja = 'sja'+ str(count)
            #href = BASE_URL_indeed + job.find('a', attrs={'id': sja}).attrs['href']
            
            
            #print(href)
#             #count = count + 1
#             # job company
#             company = job.find('span', {'class': 'company'}).text.strip()
#             #print(company)
            
            
            ### html.parser - Felipe
            r = requests.get(link)
            soup_html = BeautifulSoup(r.text, "html.parser")
            #print(soup_html)

            # Job Requirements
            jobrequirements = Job_requirements(soup_html)
            
            # Company Name 
            company = Company_Name(soup_html)           
            
            # City
            city = City(soup_html)
            
            # Province
            province = Province(soup_html)
            
            # salary
            salary = Salary(soup_html)

            
            # job type
            jobtype = Job_Type(soup_html)

            # position
            position = Position(soup_html)

            
        except:
            continue

        df_base = df_base.append({'ix': ix,
                                  'Job_ID': id,
                                  'JobTitle': input_job,
                                  'JobType':jobtype,
                                  'CompanyName': company,
                                  'Salary' : salary,
                                  'Position':position,
                                  'City': city,
                                  'Province': province,
                                  'JobRequirements': jobrequirements,
                                  'Date': now_str,
                                  'From':"Indeed",
                                  'JobLink':'link'},ignore_index=True)

# df_base


In [14]:

df_base.to_excel('out.xlsx')

In [15]:
############# READING DATAFRAME AND SCRAPING JOB DESCRIPTION ############

In [16]:
#df_base.JobRequirements.values

In [41]:
skill_source = pd.read_excel('ds.xlsx', sheet_name='skill')
kw = pd.read_excel('ds_kw.xlsx')


In [42]:

# Functions


In [43]:
df_skill_aux = pd.DataFrame(columns=['Job_ID', 'KeyWord', 'Skill_ID'])


In [44]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID


In [45]:
z = 0
for index, row in df_base.iterrows():
    k = kw.loc[kw.JobTitle == row["JobTitle"], ['KeyWord', 'Skill_ID']]
    jobReq = row["JobRequirements"]
    job_ID = row["Job_ID"]

    for idx, skills in k.iterrows():
        skill = ' ' + skills["KeyWord"]
        skill_ID = skills["Skill_ID"]
        func = find_skill(skill, jobReq)
        if func:
            # print(job_ID, skill, skill_ID)
            df_skill_aux.loc[z] = [job_ID, skill, skill_ID]
            z += 1




In [46]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID
0,107c969e3deb11c3,SQL,2
1,107c969e3deb11c3,Big Data,3
2,107c969e3deb11c3,Math,6
3,107c969e3deb11c3,Statistics,8
4,107c969e3deb11c3,Machine Learning,9
5,107c969e3deb11c3,R,11
6,107c969e3deb11c3,Master,13
7,107c969e3deb11c3,Masters,13
8,107c969e3deb11c3,Hadoop,22
9,107c969e3deb11c3,Java,31


In [48]:
# Creating df_tmp
df_tmp = pd.DataFrame()


In [49]:
#df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='ID')
df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='Job_ID')
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID
0,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",SQL,2
1,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Big Data,3
2,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6
3,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Statistics,8
4,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Machine Learning,9


In [50]:
############# Treating null values #################
values = {'Skill_ID': '-1', 'KeyWord': "Not found"}
df_tmp = df_tmp.fillna(value=values)

In [51]:
df_tmp['Skill_ID'] = df_tmp['Skill_ID'].astype(int)

In [52]:
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID
0,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",SQL,2
1,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Big Data,3
2,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6
3,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Statistics,8
4,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Machine Learning,9


In [53]:
skill_source

,Skill_ID,Skill,JobTitle,Category
0,-1,Not Identified,Data Scientist,Not Identified
1,1,Python,Data Scientist,Hard
2,2,SQL,Data Scientist,Hard
3,3,BigData,Data Scientist,Hard
4,4,NumPy,Data Scientist,Hard
5,5,Spring,Java Developer,Hard
6,6,Mathematcs,Data Scientist,Education
7,7,SQL,Java Developer,Hard
8,8,Statistics,Data Scientist,Major
9,9,Machine Learning,Data Scientist,Hard


In [54]:
####### Use this in case we want to filter jobs that does not match with keywords

#filters Skill_ID not identified
#df_filtered = df_tmp.query('Skill_ID>0')
#df_filtered.head()



In [55]:
#drop description colum to avoid a big data frame
df_tmp.drop(columns=['JobRequirements'])

,City,CompanyName,Date,From,JobLink,JobTitle,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID
0,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",SQL,2
1,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Big Data,3
2,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6
3,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Statistics,8
4,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Machine Learning,9
5,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",R,11
6,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Master,13
7,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Masters,13
8,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Hadoop,22
9,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Java,31


In [56]:
df_jobs = df_tmp.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')
#df_jobs = df_filtered.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')

In [57]:
df_jobs

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle_x,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID,Skill,JobTitle_y,Category
0,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",SQL,2,SQL,Data Scientist,Hard
1,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Big Data,3,BigData,Data Scientist,Hard
2,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Data Scientist,Education
3,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Statistics,8,Statistics,Data Scientist,Major
4,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Machine Learning,9,Machine Learning,Data Scientist,Hard
5,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",R,11,R,Data Scientist,Hard
6,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Master,13,Master,Data Scientist,Education
7,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Masters,13,Master,Data Scientist,Education
8,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Hadoop,22,Hadoop,Data Scientist,Hard
9,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Java,31,Java,Data Scientist,Hard


In [58]:
df_jobs = df_jobs.drop_duplicates(subset={'Job_ID','Skill_ID'}, keep='first', inplace=False)

In [59]:
df_jobs.head(20)

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle_x,JobType,Job_ID,Position,Province,Salary,ix,KeyWord,Skill_ID,Skill,JobTitle_y,Category
0,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",SQL,2,SQL,Data Scientist,Hard
1,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Big Data,3,BigData,Data Scientist,Hard
2,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Math,6,Mathematcs,Data Scientist,Education
3,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Statistics,8,Statistics,Data Scientist,Major
4,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Machine Learning,9,Machine Learning,Data Scientist,Hard
5,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",R,11,R,Data Scientist,Hard
6,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Master,13,Master,Data Scientist,Education
8,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Hadoop,22,Hadoop,Data Scientist,Hard
9,Toronto,Loblaw Digital,11/21/2018,Indeed,link,Loblaw Digital is the team responsible for bui...,Data Scientist,Not Informed,107c969e3deb11c3,Data Scientist,ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Java,31,Java,Data Scientist,Hard
980,Mississauga,Samsung Electronics,11/21/2018,Indeed,link,Position Summary\n\nWe are looking for a Data ...,Data Scientist,Not Informed,84a3c0cde905d6bb,"Manager, Data Scientist",ON,Not Informed,"<bound method Tag.index of <div class=""jobsear...",Python,1,Python,Data Scientist,Hard


In [60]:
df_final=df_jobs.to_excel('final_out.xlsx')